<a href="https://colab.research.google.com/github/doodeck/popes-spark/blob/main/Popes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Complete list of Popes Data Analysis

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 43.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=d87752fa5c8c36e5c45b7cbfd2d7519346ecdc0912d146b10d91e7591bd8bd29
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
!(mkdir data ; cd data ; wget "https://raw.githubusercontent.com/doodeck/popes-spark/main/data/popes.csv" )
!ls -ltraR data/

--2020-10-25 07:49:28--  https://raw.githubusercontent.com/doodeck/popes-spark/main/data/popes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38162 (37K) [text/plain]
Saving to: ‘popes.csv’

popes.csv           100%[===================>]  37.27K  --.-KB/s    in 0.01s   

2020-10-25 07:49:28 (2.57 MB/s) - ‘popes.csv’ saved [38162/38162]

.:
total 20
drwxr-xr-x 1 root root 4096 Oct 14 16:31 sample_data
drwxr-xr-x 1 root root 4096 Oct 14 16:32 .config
drwxr-xr-x 1 root root 4096 Oct 25 07:34 ..
drwxr-xr-x 1 root root 4096 Oct 25 07:49 .
drwxr-xr-x 2 root root 4096 Oct 25 07:49 data

./sample_data:
total 55512
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root  1706430 Oct 14 1

In [4]:
from pyspark.sql import (SparkSession, functions as F)

# Creates a session on a local master
spark = SparkSession.builder.appName("Orders analytics") \
    .master("local[*]").config("spark.ui.port", "4050").getOrCreate()
spark

In [5]:
# set up a tunnel to spark UI
!node --version
cmd = 'npx localtunnel --port 4050 > tunnel.txt &'
get_ipython().system_raw(cmd)
!sleep 10
!cat tunnel.txt

v12.14.1
your url is: https://green-owl-89.loca.lt


In [14]:
def main(spark):
    path = "./data/"
    filename = "popes.csv"
    # Reads a CSV file with header, called orders.csv,
    # stores it in a dataframe
    data = spark.read.format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .load(path + filename)
    # TODO: impose schema with numbers, dates, etc.

    # df.cache() # creates an entry in Spark storage
    data.show()
    data.printSchema()

    #purify the column names

    # df = data.selectExpr("Name as name", "askdaosdka as age")
    df = data.selectExpr(
        "Pontificate as pontificate",
        "'Start date' as startDate",
        "'End date' as endDate",
        "'Years and days of Pontificate' as yearsAndDays",
        "'Number of days of Pontificate (max)' as days",
        "'English name' as engName",
        "'Regnal name (latin)' as regName",
        "'Personal name' as persName",
        "'Place of birth' as placeBirth",
        "'Modern-day country of birth' as countryBirth",
        "'Age at start of papacy' as ageStart",
        "'Age at end of papacy' as ageEnd",
        "'Comments' as comments"
    )

    '''
    |-- English name: string (nullable = true)
    |-- Regnal name (latin): string (nullable = true)
    |-- : string (nullable = true)
    |-- : string (nullable = true)
    |-- : string (nullable = true)
    |-- : string (nullable = true)
    |-- : string (nullable = true)
    |-- : string (nullable = true)
    '''

    # Analyse with SQL

    df.createOrReplaceTempView("popes")

    sql_df = spark.sql('''
      desc popes
    ''')
    sql_df.show(20)


# setting log level, update this as per your requirement
spark.sparkContext.setLogLevel("warn")

main(spark)


+--------------------+----------+--------+-----------------------------+-----------------------------------+--------------------+-------------------+--------------------+--------------------+---------------------------+----------------------+--------------------+--------------------+
|         Pontificate|Start date|End date|Years and days of Pontificate|Number of days of Pontificate (max)|        English name|Regnal name (latin)|       Personal name|      Place of birth|Modern-day country of birth|Age at start of papacy|Age at end of papacy|            Comments|
+--------------------+----------+--------+-----------------------------+-----------------------------------+--------------------+-------------------+--------------------+--------------------+---------------------------+----------------------+--------------------+--------------------+
|          30 - 64/67|        30|      64|               34 to 37 years|                              13505|           St. Peter|             PET